In [1]:
from blpapi import *
import json
import pandas as pd
import ipywidgets
from pprint import pprint,pformat
import logging as log
from datetime import datetime,timedelta, date, time

In [2]:
log.basicConfig(
    level=log.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        log.FileHandler('bloomberg_api.log',mode='w'),  # Log to file
    ]
)
logger = log.getLogger(__name__)

In [3]:
df_source = []

In [4]:
config = json.load(open('bpipe_config.local.json'))
pprint(config)

APP_NAME = config["appname"]

{'appname': 'blp-apac:test',
 'hosts': [{'addr': 'jpn.cloudpoint.bloomberg.com', 'port': 8194},
           {'addr': 'hkg.cloudpoint.bloomberg.com', 'port': 8194}],
 'tlsInfo': {'password': '63IwMSyeYQsquRcC',
             'pk12path': './certs/44E7465DAD0503DB07284BA1E39DCFB8.pk12',
             'pk7path': './certs/rootCertificate.pk7'}}


In [5]:
sessOpts = SessionOptions()

for i,v in enumerate(config["hosts"]):
    sessOpts.setServerAddress(v["addr"],v["port"],i)

In [6]:
authOpts = AuthOptions.createWithApp(APP_NAME)
sessOpts.setSessionIdentityOptions(authOpts,correlationId=CorrelationId(APP_NAME))

<blpapi.internals.CorrelationId; proxy of <Swig Object of type 'blpapi_CorrelationId_t *' at 0x10c560d80> >

In [7]:
if "tlsInfo" in config:
    tlsInfo = config["tlsInfo"]
    pk12Blob = None
    pk7Blob = None
    with open(tlsInfo['pk12path'], 'rb') as pk12File:
        pk12Blob = pk12File.read()
    with open(tlsInfo['pk7path'], 'rb') as pk7File:
        pk7Blob = pk7File.read()

    sessOpts.setTlsOptions(TlsOptions.createFromBlobs(pk12Blob, tlsInfo['password'], pk7Blob))

In [8]:
def onEvent(event,session):
    global df_source

    eventType = event.eventType()
    
    for msg in event:
        msgType = msg.messageType()
        if msg.correlationId():
            corrVal = msg.correlationId().value()

        if eventType == Event.SUBSCRIPTION_DATA:
            logger.info(pformat(msg.toPy()))
        elif eventType == Event.RESPONSE or eventType == Event.PARTIAL_RESPONSE:
            _data = {}
            
            
            for _d in msg.toPy()["DATA"]:
                
                for k, v in _d.items():
                    # Handle datetime.date objects
                    if isinstance(v, date):
                        _data[k] = v.strftime('%Y/%m/%d')
                    elif isinstance(v, time):
                        _data[k] = v.strftime('%H:%M:%S.%f')[:-3]
                    else:
                        _data[k] = v

            logger.info(pformat(_data))
            df_source.append(_data)
            
        else:
            logger.info(pformat(msg.toPy()))

In [9]:
session = Session(sessOpts,onEvent)

In [10]:
session.start()

True

In [11]:
session.openService("//blp/mktlist")
session.openService("//blp/fo-discovery")

True

In [12]:
sublist = SubscriptionList()
secs = "IBM US Equity".split(",")

for s in secs:
    logger.info(f"subscribing to /chain/bpkbl/{s}")
    sublist.add(f"//blp/mktlist/chain/bpkbl/{s}",correlationId=CorrelationId(s))

session.subscribe(sublist)

In [13]:
for i in range(0,sublist.size()):
    session.cancel(sublist.correlationIdAt(i))

del sublist

In [40]:
svc = session.getService("//blp/fo-discovery")
req = svc.createRequest("OptionsScreenRequest") 
#req = svc.createRequest("FuturesScreenRequest")
underlying = req.getElement("SEARCH_CRITERIA").getElement("UNDERLYING")
underlying.setElement("UNDERLYING_SECURITY","NKY Index")
underlying.setElement("UNDERLYING_TYPE","PARSEKYABLE_DES_SOURCE")

current_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%d')

fieldFields = req.getElement("FILTER_FIELDS")
fieldFields.setElement("OPT_EXPIRE_DT_GTEQ",current_date)
fieldFields.setElement("OPT_EXPIRE_DT_LTEQ",end_date)
fieldFields.setElement("OPT_PUT_CALL","C")

logger.info(req)
df_source = []
session.sendRequest(req)

<blpapi.internals.CorrelationId; proxy of <Swig Object of type 'blpapi_CorrelationId_t *' at 0x114b1acd0> >

In [43]:
df = pd.DataFrame(df_source).sort_values(["OPT_EXPIRE_DT"],ascending=False)
df["PARSEKYABLE_DES_SOURCE,FEED_SOURCE,OPT_STRIKE_PX,OPT_EXPIRE_DT,OPT_PUT_CALL,OPT_UNDL_TICKER,FEED_EID1,FEED_EID2,FEED_EID3,FEED_EID4".split(",")]

,PARSEKYABLE_DES_SOURCE,FEED_SOURCE,OPT_STRIKE_PX,OPT_EXPIRE_DT,OPT_PUT_CALL,OPT_UNDL_TICKER,FEED_EID1,FEED_EID2,FEED_EID3,FEED_EID4
0,NKY 3 C29500 Index,JO,29500.0,2025/03/14,C,NKY,14021,14021,42684,0
2,NKYM 3 C38125 Index,JO,38125.0,2025/03/14,C,NKYM,14021,14021,42684,0
4,NKY 3 C38875 Index,JO,38875.0,2025/03/14,C,NKY,14021,14021,42684,0
6,NKY 3 C37000 Index,JO,37000.0,2025/03/14,C,NKY,14021,14021,42684,0
1,NKYM 2 C43125 Index,cJO,43125.0,2025/02/14,C,NKYM,14021,14021,42684,0
3,NKYM 2 C43250 Index,cJO,43250.0,2025/02/14,C,NKYM,14021,14021,42684,0
5,NKY 2 C42125 Index,cJO,42125.0,2025/02/14,C,NKY,14021,14021,42684,0
7,NKYM 2 C42875 Index,cJO,42875.0,2025/02/14,C,NKYM,14021,14021,42684,0
